In [1]:
from PIL import Image
from torchvision.models.optical_flow import raft_large
import os
import shutil
# from utils.file_io import read_event_h5
import h5py
import numpy as np
import glob

# def read_event_h5(file):
# #     file = h5py.File(path, 'r')
#     length = len(file['x'])
#     events = np.zeros([length, 4], dtype=np.float32)
#     events[:, 0] = file['x']
#     events[:, 1] = file['y']
#     events[:, 2] = file['t']
#     events[:, 3] = file['p']
#     file.close()
#     return events

def write_flo(flow, filename):
    """
    write optical flow in Middlebury .flo format
    :param flow: optical flow map
    :param filename: optical flow file path to be saved
    :return: None
    """
#     flow = flow[0, :, :, :]
    flow_np = flow.detach().cpu().numpy()
    f = open(filename, 'wb')
    magic = np.array([202021.25], dtype=np.float32)
    height, width = flow_np.shape[:2]
    magic.tofile(f)
    np.int32(width).tofile(f)
    np.int32(height).tofile(f)
    data = np.float32(flow_np).flatten()
    data.tofile(f)
    f.close() 
    

def preprocess(batch):
    transforms = T.Compose(
        [
            T.ConvertImageDtype(torch.float32),
            T.Normalize(mean=0.5, std=0.5),  # map [0, 1] into [-1, 1]
#             T.Resize(size=(520, 960)),
        ]
    )
    batch = transforms(batch)
    return batch

phase = 'test'
h5_directory = f'/data/GOPRO_joowan_raw/{phase}'
ref_directory = '/data/gopro_dceiflow_prev'
output_directory = f'/data/gopro_dceiflow_joowan/{phase}'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

scene_names = [filename[:-3] for filename in os.listdir(h5_directory) if filename.endswith('.h5')]

# print(h5_files)

# # 파일 이름만 리스트에 담음
# h5_filenames = [os.path.basename(file) for file in h5_files]
print(scene_names)

/root/anaconda3/envs/dcei_flow/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['GOPR0396_11_00', 'GOPR0384_11_00', 'GOPR0868_11_00', 'GOPR0410_11_00', 'GOPR0385_11_01', 'GOPR0862_11_00', 'GOPR0384_11_05', 'GOPR0881_11_01', 'GOPR0871_11_00', 'GOPR0869_11_00', 'GOPR0854_11_00']


In [2]:
# tmp = '/data/gopro_dceiflow_prev/events_train/GOPR0384_11_03/000090-event.h5'
# with h5py.File(tmp, 'r') as tmpfile:
#     print(tmpfile.keys())
#     tmparr = np.array(tmpfile['t'])
#     print(tmparr.shape)

In [3]:
# d1 = '/data1/ohjinjin/nas_ohjinjin/GOPRO_EFNet_original/GOPRO/train'
# d2 = '/data1/ohjinjin/nas_ohjinjin/GOPRO_raw_events/GOPRO_rawevents/train'
with h5py.File(os.path.join('/data/GOPRO_joowan_raw/train', 'GOPR0384_11_03.h5'), 'r') as tmpfile:
    print(tmpfile.keys())
#     tmparr = np.array(tmpfile['t'])
#     print(tmparr.shape)

<KeysViewHDF5 ['events', 'images', 'sharp_images', 'sharp_images_next', 'sharp_images_prev', 'synthesized_images']>


In [ ]:


# def binary_search_h5_dset(dset, x, l=None, r=None, side='left'):
#     """
#     Binary search for a timestamp in an HDF5 event file, without
#     loading the entire file into RAM
#     @param dset The HDF5 dataset
#     @param x The timestamp being searched for
#     @param l Starting guess for the left side (0 if None is chosen)
#     @param r Starting guess for the right side (-1 if None is chosen)
#     @param side Which side to take final result for if exact match is not found
#     @returns Index of nearest event to 'x'
#     """
#     l = 0 if l is None else l
#     r = len(dset)-1 if r is None else r
#     while l <= r:
#         mid = l + (r - l)//2;
#         midval = dset[mid]
#         if midval == x:
#             return mid
#         elif midval < x:
#             l = mid + 1
#         else:
#             r = mid - 1
#     if side == 'left':
#         return l
#     return r

# def find_ts_index(timestamp):
#     idx = binary_search_h5_dset(h5_file['events/ts'], timestamp)
#     return idx

# def compute_frame_center_indeices():
#     """
#     For each frame, find the start and end indices of the events around the
#     frame, the start and the end are at the middle between the frame and the 
#     neighborhood frames
#     """
#     frame_indices = []
#     start_idx = find_ts_index((frame_ts[0]+frame_ts[1])/2)
#     for i in range(1, len(frame_ts)-1): 
#         end_idx = find_ts_index((frame_ts[i]+frame_ts[i+1])/2)
#         frame_indices.append([start_idx, end_idx])
#         start_idx = end_idx
#     return frame_indices

# h5_file = file

# frame_ts = []
# for img_name in h5_file['images']:
#     frame_ts.append(h5_file['images/{}'.format(img_name)].attrs['timestamp'])
# event_indices = compute_frame_center_indeices()



# def get_event_indices(self, index):
#         """
#         Get start and end indices of events at index
#         @param Desired data index
#         @returns Start and end indices of events at index
#         """
#         idx0, idx1 = self.event_indices[index]
#         if not (idx0 >= 0 and idx1 <= self.num_events):
#             raise Exception("WARNING: Event indices {},{} out of bounds 0,{}".format(idx0, idx1, self.num_events))
#         return int(idx0), int(idx1)

# def get_events(self, idx0, idx1):
#     xs = self.h5_file['events/xs'][idx0:idx1]
#     ys = self.h5_file['events/ys'][idx0:idx1]
#     ts = self.h5_file['events/ts'][idx0:idx1]
#     ps = self.h5_file['events/ps'][idx0:idx1] * 2.0 - 1.0  # -1 and 1
#     return xs, ys, ts, ps

# def __getitem__(self, index, seed=None):
#     """
#     Get data at index.
#     @param index Index of data
#     @param seed Random seed for data augmentation 用于random crop等
#     @returns Dict with desired outputs (voxel grid, events, frames etc)
#         as set in constructor
#     """
#     if index < 0 or index >= self.__len__():
#         raise IndexError
#     seed = random.randint(0, 2 ** 32) if seed is None else seed

#     idx0, idx1 = self.get_event_indices(index) # the start and end index of the selected events
#     # print('DEBUG: idx0:{}, idx1:{}'.format(idx0, idx1))
#     xs, ys, ts, ps = self.get_events(idx0, idx1) # the selected events, determined by the voxel method
#     xs, ys, ts, ps = self.preprocess_events(xs, ys, ts, ps)
#     ts_0, ts_k  = ts[0], ts[-1]
#     dt = ts_k-ts_0

#     item = {'data_source_idx': self.data_source_idx, 'data_path': self.data_path,
#             'timestamp': ts_k, 'dt_between_frames': dt, 'ts_idx0': ts_0, 'ts_idx1': ts_k,
#             'idx0': idx0, 'idx1': idx1}
#     xs = torch.from_numpy(xs.astype(np.float32))
#     ys = torch.from_numpy(ys.astype(np.float32))
#     # ts = torch.from_numpy((ts-ts_0).astype(np.float32)) # ts start from 0
#     ts = torch.from_numpy(ts.astype(np.float32)) # !
#     ps = torch.from_numpy(ps.astype(np.float32))

#     num_event_frame = list(xs.shape)[0]

In [8]:
# import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import torch
import torchvision.transforms as T


for scene in scene_names:
    print(f"Current scene is {scene}")
    h5_file_path = os.path.join(h5_directory, scene+".h5")
    with h5py.File(h5_file_path, 'r') as file:
#         print(file.keys())
#         data_idx = file['images'].keys()
        
        png_files = [pngfile for pngfile in os.listdir(os.path.join(ref_directory, phase, scene,'sharp')) if pngfile.endswith('.png')]
        sorted_png_files = sorted(png_files, key=lambda x: int(os.path.splitext(x)[0]))
        smallest_number = int(os.path.splitext(sorted_png_files[0])[0])
        largest_number = int(os.path.splitext(sorted_png_files[-1])[0])

        print("가장 작은 숫자:", smallest_number)
        print("가장 큰 숫자:", largest_number)
        #     print(smallest_number+len(sorted_png_files)-1)

        for i in range(smallest_number,largest_number-1):
#             image_path_1 = f"{ref_directory}/{phase}/{scene}/sharp/{i:06d}.png"
#             image_path_2 = f"{ref_directory}/{phase}/{scene}/sharp/{i + 2:06d}.png"
#             blur_image_path = f"{ref_directory}/{phase}/{scene}/blur/{i + 1:06d}.png"
#             sharp_image_path = f"{ref_directory}/{phase}/{scene}/sharp/{i + 1:06d}.png"
#             image_1 = np.array(Image.open(image_path_1))
#             image_2 = np.array(Image.open(image_path_2))
#             blur_image = np.array(Image.open(blur_image_path))
#             sharp_image = np.array(Image.open(sharp_image_path))
            
#             plt.title(f'blur {blur_image.shape}')
#             plt.imshow(blur_image)
#             plt.show()
            
#             plt.title(f'sharp {sharp_image.shape}')
#             plt.imshow(sharp_image)
#             plt.show()
            
#             plt.title(f'sharp_prev {image_1.shape}')
#             plt.imshow(image_1)
#             plt.show()
            
#             plt.title(f'sharp_next {image_2.shape}')
#             plt.imshow(image_2)
#             plt.show()

#         for each in data_idx:
            blur = np.transpose(np.array(file[f'images/image{i-smallest_number:09d}']),(1,2,0))[:, :, [2, 1, 0]]
            syn_blur = np.transpose(np.array(file[f'synthesized_images/image{i-smallest_number:09d}']),(1,2,0))[:, :, [2, 1, 0]]
            sharp = np.transpose(np.array(file[f'sharp_images/image{i-smallest_number:09d}']),(1,2,0))[:, :, [2, 1, 0]]
            sharp_prev = np.transpose(np.array(file[f'sharp_images_prev/image{i-smallest_number:09d}']),(1,2,0))[:, :, [2, 1, 0]]
            sharp_next = np.transpose(np.array(file[f'sharp_images_next/image{i-smallest_number:09d}']),(1,2,0))[:, :, [2, 1, 0]]
            events = np.array(file[f'events/event{i-smallest_number:09d}'])################
#             plt.title(f'blur {blur.shape}')
#             plt.imshow(blur)
#             plt.show()
            
#             plt.title(f'syn_blur {syn_blur.shape}')
#             plt.imshow(syn_blur)
#             plt.show()
            
#             plt.title(f'sharp {sharp.shape}')
#             plt.imshow(sharp)
#             plt.show()
            
#             plt.title(f'sharp_prev {sharp_prev.shape}')
#             plt.imshow(sharp_prev)
#             plt.show()
            
#             plt.title(f'sharp_next {sharp_next.shape}')
#             plt.imshow(sharp_next)
#             plt.show()
            
#             print(np.array_equal(blur, blur_image))
#             print(np.array_equal(sharp, sharp_image))
#             print(np.array_equal(image_1, sharp_prev))
#             print(np.array_equal(image_2, sharp_next))
            
            # opticalflow estimation
            tensor_image_1 = F.to_tensor(sharp_prev)
            tensor_image_2 = F.to_tensor(sharp_next)
            img1_batch = tensor_image_1.unsqueeze(0)
            img2_batch = tensor_image_2.unsqueeze(0)
            
            output_blur_image_path = os.path.join(output_directory, phase, f"{scene}_{i+1:06d}-img_0.png")
            output_sharp_image_path = os.path.join(output_directory, phase, "sharp", f"{scene}_{i+1:06d}.png")
            output_second_image_path = os.path.join(output_directory, phase, f"{scene}_{i+1:06d}-img_1.png")
            output_flow01_path = os.path.join(output_directory, phase, f"{scene}_{i+1:06d}-flow_01.flo")
            output_flow10_path = os.path.join(output_directory, phase, f"{scene}_{i+1:06d}-flow_10.flo")
            output_event_path = os.path.join(output_directory, "events_"+phase, f"{scene}_{i+1:06d}-event.h5")
            if not os.path.exists(os.path.join(output_directory, phase)):
                os.makedirs(os.path.join(output_directory, phase))
            if not os.path.exists(os.path.join(output_directory, phase, "sharp")):
                os.makedirs(os.path.join(output_directory, phase, "sharp"))
            if not os.path.exists(os.path.join(output_directory, "events_"+phase)):
                os.makedirs(os.path.join(output_directory, "events_"+phase))
            
            # NumPy 배열을 PIL 이미지로 변환 후 png 파일로 저장..
            Image.fromarray(syn_blur.astype(np.uint8)).save(output_blur_image_path)
            Image.fromarray(sharp.astype(np.uint8)).save(output_sharp_image_path)
            Image.fromarray(sharp_next.astype(np.uint8)).save(output_second_image_path)

            # 누적 이벤트 .h5 저장
            
            # HDF5 파일로 저장
            with h5py.File(output_event_path, 'w') as ev_f:
                ev_f.create_dataset('x', data=events[:, 0])
                ev_f.create_dataset('y', data=events[:, 1])
                ev_f.create_dataset('t', data=events[:, 2])
                ev_f.create_dataset('p', data=events[:, 3])
            
            # If you can, run this example on a GPU, it will be a lot faster.
            device = "cuda" if torch.cuda.is_available() else "cpu"
            img1_batch = preprocess(img1_batch).to(device)
            img2_batch = preprocess(img2_batch).to(device)

            model = raft_large(pretrained=True, progress=False).to(device)
            model = model.eval()

            list_of_flows = model(img1_batch.to(device), img2_batch.to(device))
            predicted_flows = list_of_flows[-1]
            # .flo 파일로 저장
            write_flo(predicted_flows[0].permute(1, 2, 0), output_flow01_path)
            list_of_flows = model(img2_batch.to(device), img1_batch.to(device))
            predicted_flows = list_of_flows[-1]

            # .flo 파일로 저장
            write_flo(predicted_flows[0].permute(1, 2, 0), output_flow10_path)
            
#             break
#         break
            
            

Current scene is GOPR0396_11_00
가장 작은 숫자: 1
가장 큰 숫자: 100


/root/anaconda3/envs/dcei_flow/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/anaconda3/envs/dcei_flow/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Raft_Large_Weights.C_T_SKHT_V2`. You can also use `weights=Raft_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Current scene is GOPR0384_11_00
가장 작은 숫자: 1
가장 큰 숫자: 100
Current scene is GOPR0868_11_00
가장 작은 숫자: 1
가장 큰 숫자: 100
Current scene is GOPR0410_11_00
가장 작은 숫자: 101
가장 큰 숫자: 234
Current scene is GOPR0385_11_01
가장 작은 숫자: 3011
가장 큰 숫자: 3110
Current scene is GOPR0862_11_00
가장 작은 숫자: 1
가장 큰 숫자: 77
Current scene is GOPR0384_11_05
가장 작은 숫자: 4001
가장 큰 숫자: 4100
Current scene is GOPR0881_11_01
가장 작은 숫자: 201
가장 큰 숫자: 300
Current scene is GOPR0871_11_00
가장 작은 숫자: 1
가장 큰 숫자: 100
Current scene is GOPR0869_11_00
가장 작은 숫자: 1
가장 큰 숫자: 100
Current scene is GOPR0854_11_00
가장 작은 숫자: 1
가장 큰 숫자: 100


In [9]:

phase = 'train'
h5_directory = f'/data/GOPRO_joowan_raw/{phase}'
ref_directory = '/data/gopro_dceiflow_prev'
output_directory = f'/data/gopro_dceiflow_joowan/{phase}'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

scene_names = [filename[:-3] for filename in os.listdir(h5_directory) if filename.endswith('.h5')]

# print(h5_files)

# # 파일 이름만 리스트에 담음
# h5_filenames = [os.path.basename(file) for file in h5_files]
print(scene_names)

['GOPR0372_07_01', 'GOPR0384_11_02', 'GOPR0384_11_01', 'GOPR0380_11_00', 'GOPR0372_07_00', 'GOPR0477_11_00', 'GOPR0881_11_00', 'GOPR0871_11_01', 'GOPR0384_11_04', 'GOPR0374_11_01', 'GOPR0386_11_00', 'GOPR0868_11_01', 'GOPR0374_11_00', 'GOPR0857_11_00', 'GOPR0384_11_03', 'GOPR0385_11_00', 'GOPR0374_11_03', 'GOPR0868_11_02', 'GOPR0378_13_00', 'GOPR0374_11_02', 'GOPR0884_11_00', 'GOPR0379_11_00']


In [10]:
# import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import torch
import torchvision.transforms as T


for scene in scene_names:
    print(f"Current scene is {scene}")
    h5_file_path = os.path.join(h5_directory, scene+".h5")
    with h5py.File(h5_file_path, 'r') as file:
#         print(file.keys())
#         data_idx = file['images'].keys()
        
        png_files = [pngfile for pngfile in os.listdir(os.path.join(ref_directory, phase, scene,'sharp')) if pngfile.endswith('.png')]
        sorted_png_files = sorted(png_files, key=lambda x: int(os.path.splitext(x)[0]))
        smallest_number = int(os.path.splitext(sorted_png_files[0])[0])
        largest_number = int(os.path.splitext(sorted_png_files[-1])[0])

        print("가장 작은 숫자:", smallest_number)
        print("가장 큰 숫자:", largest_number)
        #     print(smallest_number+len(sorted_png_files)-1)

        for i in range(smallest_number,largest_number-1):
#             image_path_1 = f"{ref_directory}/{phase}/{scene}/sharp/{i:06d}.png"
#             image_path_2 = f"{ref_directory}/{phase}/{scene}/sharp/{i + 2:06d}.png"
#             blur_image_path = f"{ref_directory}/{phase}/{scene}/blur/{i + 1:06d}.png"
#             sharp_image_path = f"{ref_directory}/{phase}/{scene}/sharp/{i + 1:06d}.png"
#             image_1 = np.array(Image.open(image_path_1))
#             image_2 = np.array(Image.open(image_path_2))
#             blur_image = np.array(Image.open(blur_image_path))
#             sharp_image = np.array(Image.open(sharp_image_path))
            
#             plt.title(f'blur {blur_image.shape}')
#             plt.imshow(blur_image)
#             plt.show()
            
#             plt.title(f'sharp {sharp_image.shape}')
#             plt.imshow(sharp_image)
#             plt.show()
            
#             plt.title(f'sharp_prev {image_1.shape}')
#             plt.imshow(image_1)
#             plt.show()
            
#             plt.title(f'sharp_next {image_2.shape}')
#             plt.imshow(image_2)
#             plt.show()

#         for each in data_idx:
            blur = np.transpose(np.array(file[f'images/image{i-smallest_number:09d}']),(1,2,0))[:, :, [2, 1, 0]]
            syn_blur = np.transpose(np.array(file[f'synthesized_images/image{i-smallest_number:09d}']),(1,2,0))[:, :, [2, 1, 0]]
            sharp = np.transpose(np.array(file[f'sharp_images/image{i-smallest_number:09d}']),(1,2,0))[:, :, [2, 1, 0]]
            sharp_prev = np.transpose(np.array(file[f'sharp_images_prev/image{i-smallest_number:09d}']),(1,2,0))[:, :, [2, 1, 0]]
            sharp_next = np.transpose(np.array(file[f'sharp_images_next/image{i-smallest_number:09d}']),(1,2,0))[:, :, [2, 1, 0]]
            events = np.array(file[f'events/event{i-smallest_number:09d}'])################
#             plt.title(f'blur {blur.shape}')
#             plt.imshow(blur)
#             plt.show()
            
#             plt.title(f'syn_blur {syn_blur.shape}')
#             plt.imshow(syn_blur)
#             plt.show()
            
#             plt.title(f'sharp {sharp.shape}')
#             plt.imshow(sharp)
#             plt.show()
            
#             plt.title(f'sharp_prev {sharp_prev.shape}')
#             plt.imshow(sharp_prev)
#             plt.show()
            
#             plt.title(f'sharp_next {sharp_next.shape}')
#             plt.imshow(sharp_next)
#             plt.show()
            
#             print(np.array_equal(blur, blur_image))
#             print(np.array_equal(sharp, sharp_image))
#             print(np.array_equal(image_1, sharp_prev))
#             print(np.array_equal(image_2, sharp_next))
            
            # opticalflow estimation
            tensor_image_1 = F.to_tensor(sharp_prev)
            tensor_image_2 = F.to_tensor(sharp_next)
            img1_batch = tensor_image_1.unsqueeze(0)
            img2_batch = tensor_image_2.unsqueeze(0)
            
            output_blur_image_path = os.path.join(output_directory, phase, f"{scene}_{i+1:06d}-img_0.png")
            output_sharp_image_path = os.path.join(output_directory, phase, "sharp", f"{scene}_{i+1:06d}.png")
            output_second_image_path = os.path.join(output_directory, phase, f"{scene}_{i+1:06d}-img_1.png")
            output_flow01_path = os.path.join(output_directory, phase, f"{scene}_{i+1:06d}-flow_01.flo")
            output_flow10_path = os.path.join(output_directory, phase, f"{scene}_{i+1:06d}-flow_10.flo")
            output_event_path = os.path.join(output_directory, "events_"+phase, f"{scene}_{i+1:06d}-event.h5")
            if not os.path.exists(os.path.join(output_directory, phase)):
                os.makedirs(os.path.join(output_directory, phase))
            if not os.path.exists(os.path.join(output_directory, phase, "sharp")):
                os.makedirs(os.path.join(output_directory, phase, "sharp"))
            if not os.path.exists(os.path.join(output_directory, "events_"+phase)):
                os.makedirs(os.path.join(output_directory, "events_"+phase))
            
            # NumPy 배열을 PIL 이미지로 변환 후 png 파일로 저장..
            Image.fromarray(syn_blur.astype(np.uint8)).save(output_blur_image_path)
            Image.fromarray(sharp.astype(np.uint8)).save(output_sharp_image_path)
            Image.fromarray(sharp_next.astype(np.uint8)).save(output_second_image_path)

            # 누적 이벤트 .h5 저장
            
            # HDF5 파일로 저장
            with h5py.File(output_event_path, 'w') as ev_f:
                ev_f.create_dataset('x', data=events[:, 0])
                ev_f.create_dataset('y', data=events[:, 1])
                ev_f.create_dataset('t', data=events[:, 2])
                ev_f.create_dataset('p', data=events[:, 3])
            
            # If you can, run this example on a GPU, it will be a lot faster.
            device = "cuda" if torch.cuda.is_available() else "cpu"
            img1_batch = preprocess(img1_batch).to(device)
            img2_batch = preprocess(img2_batch).to(device)

            model = raft_large(pretrained=True, progress=False).to(device)
            model = model.eval()

            list_of_flows = model(img1_batch.to(device), img2_batch.to(device))
            predicted_flows = list_of_flows[-1]
            # .flo 파일로 저장
            write_flo(predicted_flows[0].permute(1, 2, 0), output_flow01_path)
            list_of_flows = model(img2_batch.to(device), img1_batch.to(device))
            predicted_flows = list_of_flows[-1]

            # .flo 파일로 저장
            write_flo(predicted_flows[0].permute(1, 2, 0), output_flow10_path)
            
#             break
#         break
            
            

Current scene is GOPR0372_07_01
가장 작은 숫자: 601
가장 큰 숫자: 675
Current scene is GOPR0384_11_02
가장 작은 숫자: 1301
가장 큰 숫자: 1400
Current scene is GOPR0384_11_01
가장 작은 숫자: 951
가장 큰 숫자: 1050
Current scene is GOPR0380_11_00
가장 작은 숫자: 134
가장 큰 숫자: 193
Current scene is GOPR0372_07_00
가장 작은 숫자: 47
가장 큰 숫자: 146
Current scene is GOPR0477_11_00
가장 작은 숫자: 1
가장 큰 숫자: 80
Current scene is GOPR0881_11_00
가장 작은 숫자: 1
가장 큰 숫자: 100
Current scene is GOPR0871_11_01
가장 작은 숫자: 181
가장 큰 숫자: 280
Current scene is GOPR0384_11_04
가장 작은 숫자: 2801
가장 큰 숫자: 2900
Current scene is GOPR0374_11_01
가장 작은 숫자: 203
가장 큰 숫자: 282
Current scene is GOPR0386_11_00
가장 작은 숫자: 247
가장 큰 숫자: 346
Current scene is GOPR0868_11_01
가장 작은 숫자: 221
가장 큰 숫자: 320
Current scene is GOPR0374_11_00
가장 작은 숫자: 1
가장 큰 숫자: 150
Current scene is GOPR0857_11_00
가장 작은 숫자: 1
가장 큰 숫자: 100
Current scene is GOPR0384_11_03
가장 작은 숫자: 2101
가장 큰 숫자: 2200
Current scene is GOPR0385_11_00
가장 작은 숫자: 101
가장 큰 숫자: 200
Current scene is GOPR0374_11_03
가장 작은 숫자: 2481
가장 큰 숫자: 252